In [2]:
# 1. Setup: Load Libraries
# This cell installs and imports all the necessary Python libraries for the analysis,
# visualization, and interactive controls.
# You only need to run this cell once per session.

# --- Install required packages ---
!pip install -q pandas numpy scipy matplotlib seaborn ipywidgets

# --- Import Libraries ---
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import PercentFormatter, FuncFormatter
import os

# --- New libraries for interactive widgets and file handling ---
import ipywidgets as widgets
from IPython.display import display, clear_output
import io

print("✅ Libraries loaded successfully. You can now proceed to the Control Panel.")


✅ Libraries loaded successfully. You can now proceed to the Control Panel.


In [3]:
# ## 2. Control Panel ⚙️
# This is the only section you need to edit. Use the controls below to
# configure your test. Once you're done, run the cells that follow.

# --- Create Interactive Widgets ---

# File Uploader Widget: Allows users to upload their data file directly.
# This is more user-friendly than manually editing a file path in the code.
uploader = widgets.FileUpload(
    accept='.xlsx',
    description='Upload Data',
    button_style='info',
    tooltip='Upload your A/B test data in .xlsx format'
)

# Risk Threshold Slider Widget: Provides an intuitive way to set risk.
# The slider is framed in plain English, which is easier to understand
# than an abstract number like 0.01.
risk_slider = widgets.FloatSlider(
    value=0.01,
    min=0.005,
    max=0.05,
    step=0.005,
    description='Risk Tolerance:',
    style={'description_width': 'initial'},
    readout_format='.2%',
    tooltip='Lower = more cautious. Higher = more aggressive.'
)

# Display the widgets
# This will render the interactive controls in the notebook output.
print("Please configure your test using the controls below:")
display(uploader, risk_slider)


Please configure your test using the controls below:


FileUpload(value=(), accept='.xlsx', button_style='info', description='Upload Data', tooltip='Upload your A/B …

FloatSlider(value=0.01, description='Risk Tolerance:', max=0.05, min=0.005, readout_format='.2%', step=0.005, …

In [4]:
# ## 3. Data Processing & Analysis 🔬
# This section contains the core logic for the analysis. You don't need to
# change anything here. Just click the "Run Analysis" button below.
# It will automatically use the inputs from the Control Panel above.

# --- Main Analysis Function ---
# This function consolidates all the complex calculations from the original
# notebook into a single, reusable block of code.
def run_full_analysis(df, prior_alpha, prior_beta, risk_threshold):
    """
    Performs a full Bayesian A/B test analysis and returns the results DataFrame.
    """
    # Configuration for the Bayesian Analysis
    N_SAMPLES = 200_000

    # 1. --- Check for Low Sample Size (Robustness Check) ---
    low_sample_threshold = 100
    if (df['reach'] < low_sample_threshold).any():
        print(f"⚠️ Warning: Low Sample Size Detected. Results may not be stable.")

    # 2. --- Calculate Posterior Parameters ---
    df['posterior_alpha'] = prior_alpha + df['conversion']
    df['posterior_beta'] = prior_beta + (df['reach'] - df['conversion'])
    df['posterior_mean'] = df['posterior_alpha'] / (df['posterior_alpha'] + df['posterior_beta'])

    # 3. --- Run Monte Carlo Simulation ---
    samples = {
        name: np.random.beta(row['posterior_alpha'], row['posterior_beta'], size=N_SAMPLES)
        for name, row in df.set_index('variant').iterrows()
    }
    samples_df = pd.DataFrame(samples)

    # 4. --- Calculate "Probability to be Best" ---
    best_variant_counts = samples_df.idxmax(axis=1).value_counts(normalize=True)
    df['prob_to_be_best'] = df['variant'].map(best_variant_counts).fillna(0)

    # 5. --- Calculate "Expected Loss" (Risk) ---
    max_conversion_rates = samples_df.max(axis=1)
    loss_df = samples_df.apply(lambda x: max_conversion_rates - x)
    expected_loss = loss_df.mean()
    df['expected_loss'] = df['variant'].map(expected_loss)

    return df.sort_values(by='expected_loss', ascending=True).reset_index(drop=True)

# --- Create a button to trigger the analysis ---
# This makes the user experience interactive and clear.
run_button = widgets.Button(description="Run Analysis", button_style='success')
output_area = widgets.Output()

def on_run_button_clicked(b):
    # This function runs when the button is clicked.
    with output_area:
        clear_output(wait=True) # Clears previous results

        # Validate that a file has been uploaded
        if not uploader.value:
            print("❌ Error: Please upload a data file in the Control Panel first.")
            return

        # Get data from the uploader widget
        uploaded_file_info = list(uploader.value.values())[0]
        content = uploaded_file_info['content']
        df = pd.read_excel(io.BytesIO(content))

        # Get risk tolerance from the slider widget
        RISK_THRESHOLD = risk_slider.value

        print("Running analysis with the following parameters:")
        print(f" - Risk Tolerance: {RISK_THRESHOLD:.2%}")
        print("---")

        # Run the full analysis using the function defined above
        global results_df # Store results globally to be used by the report cell
        results_df = run_full_analysis(df, 0.5, 0.5, RISK_THRESHOLD)

        print("✅ Analysis Complete. View the report in the next section.")
        # Display a preview of the results table
        display(results_df[['variant', 'prob_to_be_best', 'expected_loss']].style.format({
            "prob_to_be_best": "{:.2%}",
            "expected_loss": "{:.4%}"
        }).set_caption("Results Summary").hide(axis="index"))

# Link the function to the button click event
run_button.on_click(on_run_button_clicked)

# Display the button and the area for its output
display(run_button, output_area)


Button(button_style='success', description='Run Analysis', style=ButtonStyle())

Output()

In [5]:
# ## 4. Automated Report 📄
# Below is the final report. It includes a clear verdict and a visual summary
# that you can save as a PNG and share with your team.

# --- Visual Report Generator ---
def create_visual_report(results_df, risk_threshold):
    """
    Generates a single, shareable PNG image summarizing the A/B test results.
    """
    # --- 1. Extract Data and Determine Verdict ---
    best_candidate = results_df.iloc[0]
    runner_up = results_df.iloc[1] if len(results_df) > 1 else None

    is_winner = best_candidate['expected_loss'] < risk_threshold
    verdict_text = f"DEPLOY VARIANT '{best_candidate['variant']}'" if is_winner else "TEST IS INCONCLUSIVE"
    verdict_color = '#2E7D32' if is_winner else '#C62828'

    # --- 2. Create the Figure Layout ---
    # We use a grid layout to arrange the different report elements.
    fig = plt.figure(figsize=(11, 8), constrained_layout=True)
    gs = fig.add_gridspec(3, 2)
    fig.suptitle("A/B Test Final Report", fontsize=22, fontweight='bold')

    # --- 3. Add Report Components to the Grid ---

    # a) Main Verdict (Top Banner)
    ax_verdict = fig.add_subplot(gs[0, :])
    ax_verdict.text(0.5, 0.5, verdict_text, ha='center', va='center', fontsize=30, fontweight='bold', color=verdict_color,
                    bbox=dict(facecolor='white', edgecolor=verdict_color, boxstyle='round,pad=0.3', lw=2))
    ax_verdict.axis('off')

    # b) Key Metrics (Middle Left)
    ax_metrics = fig.add_subplot(gs[1, 0])
    ax_metrics.axis('off')
    ax_metrics.set_title("Key Metrics", fontsize=16, fontweight='bold', loc='left')

    prob_best_str = f"{best_candidate['prob_to_be_best']:.1%}"
    risk_str = f"{best_candidate['expected_loss']:.3%}"
    metrics_text = f"Winner: \t\t{best_candidate['variant']}\n\n" \
                   f"Prob. to be Best: \t{prob_best_str}\n\n" \
                   f"Risk (Expected Loss): \t{risk_str}"

    if is_winner and runner_up is not None:
        uplift = (best_candidate['posterior_mean'] - runner_up['posterior_mean']) / runner_up['posterior_mean']
        metrics_text += f"\n\nExpected Uplift: \t+{uplift:.1%}"

    ax_metrics.text(0.0, 0.4, metrics_text, ha='left', va='center', fontsize=12, family='monospace')

    # c) Posterior Plot (Right Side)
    ax_plot = fig.add_subplot(gs[1:, 1])
    sorted_plot_df = results_df.sort_values('posterior_mean', ascending=False)
    colors = plt.cm.viridis(np.linspace(0.1, 0.9, len(sorted_plot_df)))
    max_x = stats.beta.ppf(0.999, sorted_plot_df['posterior_alpha'].max(), sorted_plot_df['posterior_beta'].min()) * 1.2
    x = np.linspace(0, max_x, 500)

    for i, (row, color) in enumerate(zip(sorted_plot_df.itertuples(), colors)):
        pdf = stats.beta.pdf(x, row.posterior_alpha, row.posterior_beta)
        ax_plot.plot(x, pdf, color=color, lw=2, label=row.variant)
        ax_plot.fill_between(x, 0, pdf, alpha=0.2, color=color)

    ax_plot.set_title("Conversion Rate Distributions", fontsize=14)
    ax_plot.legend()
    ax_plot.spines[['top', 'right']].set_visible(False)
    ax_plot.yaxis.set_ticks([])

    # d) Stakeholder Summary (Bottom Left)
    ax_summary = fig.add_subplot(gs[2, 0])
    ax_summary.axis('off')
    ax_summary.set_title("Summary for Stakeholders", fontsize=16, fontweight='bold', loc='left')

    if is_winner:
        summary_text = f"The analysis confidently recommends deploying Variant '{best_candidate['variant']}'.\n\nIt has the highest chance of being the best option, and the risk of choosing it is well below our safety limit."
    else:
        summary_text = f"The results are not yet clear enough to make a confident decision.\n\nOur best option still has a risk level higher than our limit. We recommend collecting more data to get a clearer winner."

    ax_summary.text(0.0, 0.4, summary_text, ha='left', va='center', fontsize=11, wrap=True)

    return fig

# --- Display the Report and Save Button ---
# This logic runs after the analysis is complete.
report_output_area = widgets.Output()
with report_output_area:
    if 'results_df' in locals():
        # Generate the visual report
        report_fig = create_visual_report(results_df, risk_slider.value)

        # Create and display the save button
        save_button = widgets.Button(description="Save Report as PNG", button_style='primary')
        save_output = widgets.Output()

        def on_save_clicked(b):
            with save_output:
                clear_output(wait=True)
                report_fig.savefig("ab_test_report.png", dpi=150, bbox_inches='tight')
                print("✅ Report saved as 'ab_test_report.png'")

        save_button.on_click(on_save_clicked)
        display(save_button, save_output)

        # Display the plot in the notebook
        plt.show(report_fig)
    else:
        print("Please run the analysis cell first to generate a report.")

display(report_output_area)


Output()